<a href="https://colab.research.google.com/github/tzf101/BDA-Bangla-Text-Data-Augmentation/blob/main/utils_notebook/pp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Libraries

### Libraries

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive", force_remount=True)

Mounted at /content/MyDrive


In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-s2d7she0
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-s2d7she0
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install sentencepiece

In [ ]:
# !pip install datsets transformers[sentencepiece]
# !pip install sentencepiece

In [ ]:
!pip install rouge

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d96e56a72dd86a73f5e022a3a03d937815cabd6ad664397defbfb28d82868191
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize
import sentencepiece as spm
import pandas as pd

import sacrebleu
from rouge import Rouge
from sacrebleu import corpus_bleu
from bert_score import score
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu

### Models

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_banglaparaphrase")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_banglaparaphrase", use_fast=False)

In [ ]:
sbert_model = SentenceTransformer('l3cube-pune/bengali-sentence-bert-nli')

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

### Paraphrase function

In [ ]:
def calculate_sbert_score(original, augmented):
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()
    return sbert_score

In [ ]:

def calculate_scores(original, augmented):
    # BLEU Score
    reference = original
    candidate = augmented
    # bleu_score = sentence_bleu(reference, candidate)
    bleu_score = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(augmented, original)]
    # BERTScore
    P, R, F1 = score([augmented], [original], lang="bn", rescale_with_baseline=True)

    # SBERT Score with Cosine Similarity
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()

    return bleu_score, F1.item(), sbert_score

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

class BanglaParaphraseGenerator:
    def __init__(self):
        self.model = model
        self.tokenizer = tokenizer

    def generateParaphrase(self, sentence, max_length=100):
        input_tokens = self.tokenizer(normalize(sentence), return_tensors="pt").input_ids
        generated_tokens = self.model.generate(input_tokens, max_length=max_length)
        decoded_tokens = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        return decoded_tokens





In [ ]:
# Define a function to apply paraphrasing
def apply_paraphrasing(row):
    return paraphrase_generator.generateParaphrase(row['original_sentence'])


In [ ]:
def paraphrase_and_evaluate_dataset(file_path, original_col_name, new_col_name):
    # Load the dataset
    df = pd.read_csv(file_path)
    df = df.rename(columns={original_col_name: 'original_sentence'})

    # Apply the text augmentation function
    df[new_col_name] = df.apply(apply_paraphrasing, axis=1)

    # Compute ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(df[new_col_name], df["original_sentence"])
    rouge_df = pd.DataFrame([
        {
            'r1f1': score['rouge-1']['f'],
            'r2f1': score['rouge-2']['f'],
            'rlf1': score['rouge-l']['f'],
        }
        for score in rouge_scores
    ])

    # Compute SacreBLEU scores
    sacrebleu_scores = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(df[new_col_name], df["original_sentence"])]
    df["sacrebleu_score"] = sacrebleu_scores

    # SBERT Score with Cosine Similarity
    df["sbert_score"] = [calculate_sbert_score(orig, aug) for orig, aug in zip(df["original_sentence"], df[new_col_name])]

    # BERTScore
    P, R, F1 = score(df[new_col_name], df["original_sentence"], lang="en", rescale_with_baseline=True)
    df["bertscore_f1"] = F1.tolist()

    # Combine the dataframes
    result_df = pd.concat([df, rouge_df], axis=1)
    result_df["method"] = "pp"

    return result_df

# Testing

In [ ]:
# Example usage:
input_text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
paraphrase_generator = BanglaParaphraseGenerator()
output_text = paraphrase_generator.generateParaphrase(input_text)
print(output_text)

সঠিক তদন্ত করা উচিত এবং সেই ব্যক্তিকে বিচারের আওতায় আনা উচিত, যিনি তা করেছেন।


# Running on Dataset

In [ ]:
file_path = '/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10.csv'

In [ ]:
result_df = paraphrase_and_evaluate_dataset(file_path, 'sentence1', 'augmented_sentence')

In [ ]:
result_df

,original_sentence,label,augmented_sentence,sacrebleu_score,sbert_score,r1p,r1r,r1f,r2p,r2r,r2f,rlp,rlr,rlf,method
0,নুসরাত কে বাংলাদেশ থেকে ভাগানো হক আমারা চাই,Negative,আমরা চাই নুসরত যেন বাংলাদেশ থেকে পালিয়ে যায়।,12.223076,0.686370,0.375000,0.375000,0.375000,0.142857,0.142857,0.142857,0.250000,0.250000,0.250000,pp
1,"মাল একখান,,,,",Negative,এটা একটা মাল।,0.000000,0.750619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pp
2,এগিয়ে যাও আমরা আছি তোমাদের সাথে,Positive,"এগিয়ে যাও, আমরা তোমার সাথে আছি।",8.643020,0.964527,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,pp
3,শালা। তুই ভুতের বাচ্চা সোলায়মান বইটি পড়েছিস? ও...,Negative,"তুমি যদি তোমার মত কামলা পড়তে পারো, তাহলে তুমি...",1.870679,0.587398,0.230769,0.096774,0.136364,0.076923,0.032258,0.045455,0.230769,0.096774,0.136364,pp
4,"সেই কবে ছিল উচ্ছাস, কিছু শঙ্কায় ভরা চুম্বন ছি...",Neutral,"সেই সময় উত্তেজনা ছিল, কিছু ভয় ভরা চুম্বন ছিল...",12.398992,0.846904,0.400000,0.461538,0.428571,0.142857,0.153846,0.148148,0.400000,0.461538,0.428571,pp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,করলাম জলদি করে কাতার থেকে আছি সব সময়,Positive,"আমি দ্রুত কাতার থেকে এসেছি, সবসময় আমি কাতার থ...",7.495553,0.777421,0.285714,0.250000,0.266667,0.125000,0.142857,0.133333,0.285714,0.250000,0.266667,pp
162,কিছুই বলার নাই ভাই !!,Neutral,"আমার কিছু বলার নেই, ভাই!",14.535768,0.896407,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,pp
163,ও মোর খোদা আমাদের commerce এর পোলা গো কোনো ভাত...,Negative,"ওহ, খোদা, আমাদের কমোডরের পোলা, কোন ভাত নেই।",4.503734,0.890798,0.125000,0.083333,0.100000,0.000000,0.000000,0.000000,0.125000,0.083333,0.100000,pp
164,হাজার বছেরে বাংলাদেশ জিতে পারবেনা বাংলা দেশ হা...,Neutral,"হাজার হাজার বছর ধরে বাংলাদেশ জিততে পারবে না, হ...",19.850824,0.804030,0.444444,0.444444,0.444444,0.157895,0.166667,0.162162,0.333333,0.333333,0.333333,pp


### Saving the dataset

In [ ]:
result_df.to_csv('/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10_pp.csv', index=False)